## 1. Imports

In [1]:
import pandas as pd
import numpy as np
import pickle

from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LinearRegression, Lasso, LassoCV
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor

import plotly.express as px

## 2. Data

In [2]:
data = pd.read_csv("data/WineQT.csv")
data.columns = data.columns.str.replace(" ", "_")

data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1143 entries, 0 to 1142
Data columns (total 13 columns):
 #   Column                Non-Null Count  Dtype  
---  ------                --------------  -----  
 0   fixed_acidity         1143 non-null   float64
 1   volatile_acidity      1143 non-null   float64
 2   citric_acid           1143 non-null   float64
 3   residual_sugar        1143 non-null   float64
 4   chlorides             1143 non-null   float64
 5   free_sulfur_dioxide   1143 non-null   float64
 6   total_sulfur_dioxide  1143 non-null   float64
 7   density               1143 non-null   float64
 8   pH                    1143 non-null   float64
 9   sulphates             1143 non-null   float64
 10  alcohol               1143 non-null   float64
 11  quality               1143 non-null   int64  
 12  Id                    1143 non-null   int64  
dtypes: float64(11), int64(2)
memory usage: 116.2 KB


## 2.1. Data preparation

In [3]:
target = data["quality"]

features = data.drop("quality", axis=1).drop("Id", axis=1)



X_train, X_test, y_train, y_test = train_test_split(features, target,
                                                    test_size = 0.2,
                                                    random_state = 23)

## 3. Modelling

In [4]:
models = [LinearRegression(), Lasso(), LassoCV(cv=5), RandomForestRegressor(), GradientBoostingRegressor()]

results = []
for model in models:
    model.fit(X_train, y_train)
    preds = model.predict(X_test)

    rmse = np.sqrt(np.mean((y_test - preds)**2))
    results.append({"model": model, "rmse": rmse})

results_df = pd.DataFrame(results)
results_df

,model,rmse
0,LinearRegression(),0.684036
1,Lasso(),0.869115
2,LassoCV(cv=5),0.695059
3,"(DecisionTreeRegressor(max_features=1.0, rando...",0.617206
4,([DecisionTreeRegressor(criterion='friedman_ms...,0.659445


In [5]:
# the random forest seems the best performing model

# hyperparameter tuning
random_grid = {"n_estimators": [5, 20, 50, 100],
               "max_features": ["sqrt", "log2"],
               "max_depth": [int(i) for i in np.linspace(10, 120, num=12)],
               "min_samples_split": [2, 6, 10],
               "min_samples_leaf": [1, 3, 4],
               "bootstrap": [True, False]}

rf_random = RandomizedSearchCV(estimator = RandomForestRegressor(), param_distributions=random_grid,
                               n_iter=100, cv=5, verbose=2, random_state=23, n_jobs=-1)

rf_random.fit(X_train, y_train)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[CV] END bootstrap=True, max_depth=70, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=20; total time=   0.0s
[CV] END bootstrap=True, max_depth=70, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=20; total time=   0.0s
[CV] END bootstrap=False, max_depth=60, max_features=log2, min_samples_leaf=3, min_samples_split=6, n_estimators=50; total time=   0.1s
[CV] END bootstrap=False, max_depth=60, max_features=log2, min_samples_leaf=3, min_samples_split=6, n_estimators=50; total time=   0.1s
[CV] END bootstrap=True, max_depth=70, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=20; total time=   0.0s
[CV] END bootstrap=True, max_depth=70, max_features=log2, min_samples_leaf=1, min_samples_split=10, n_estimators=20; total time=   0.0s
[CV] END bootstrap=False, max_depth=60, max_features=log2, min_samples_leaf=3, min_samples_split=6, n_estimators=50; total time=   0.1s
[CV] END bootstrap=False, max_depth=120, max_fea

RandomizedSearchCV(cv=5, estimator=RandomForestRegressor(), n_iter=100,
                   n_jobs=-1,
                   param_distributions={'bootstrap': [True, False],
                                        'max_depth': [10, 20, 30, 40, 50, 60,
                                                      70, 80, 90, 100, 110,
                                                      120],
                                        'max_features': ['sqrt', 'log2'],
                                        'min_samples_leaf': [1, 3, 4],
                                        'min_samples_split': [2, 6, 10],
                                        'n_estimators': [5, 20, 50, 100]},
                   random_state=23, verbose=2)

In [6]:
# get best parameters
best_params = rf_random.best_params_
best_params

{'n_estimators': 100,
 'min_samples_split': 2,
 'min_samples_leaf': 1,
 'max_features': 'log2',
 'max_depth': 90,
 'bootstrap': True}

In [7]:
# refit the model with the best parameters
model = RandomForestRegressor(n_estimators=best_params.get("n_estimators"),
                              min_samples_split=best_params.get("min_samples_split"),
                              min_samples_leaf=best_params.get("min_samples_leaf"),
                              max_features=best_params.get("max_features"),
                              max_depth=best_params.get("max_depth"),
                              bootstrap=best_params.get("bootstrap"),
                              random_state=23)

model.fit(X_train, y_train)

preds = model.predict(X_test)

rmse = np.sqrt(np.mean((y_test - preds)**2))

rmse

0.621831005279278

In [8]:
# Visualise feature importance (this could be added as a separate tab to understand the model)

forest_importances = pd.Series(model.feature_importances_, index=X_train.columns)
px.bar(x=forest_importances.index, y=forest_importances)

In [9]:
data.describe()

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol,quality,Id
count,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000,1143.000000
mean,8.311111,0.531339,0.268364,2.532152,0.086933,15.615486,45.914698,0.996730,3.311015,0.657708,10.442111,5.657043,804.969379
std,1.747595,0.179633,0.196686,1.355917,0.047267,10.250486,32.782130,0.001925,0.156664,0.170399,1.082196,0.805824,463.997116
min,4.600000,0.120000,0.000000,0.900000,0.012000,1.000000,6.000000,0.990070,2.740000,0.330000,8.400000,3.000000,0.000000
25%,7.100000,0.392500,0.090000,1.900000,0.070000,7.000000,21.000000,0.995570,3.205000,0.550000,9.500000,5.000000,411.000000
50%,7.900000,0.520000,0.250000,2.200000,0.079000,13.000000,37.000000,0.996680,3.310000,0.620000,10.200000,6.000000,794.000000
75%,9.100000,0.640000,0.420000,2.600000,0.090000,21.000000,61.000000,0.997845,3.400000,0.730000,11.100000,6.000000,1209.500000
max,15.900000,1.580000,1.000000,15.500000,0.611000,68.000000,289.000000,1.003690,4.010000,2.000000,14.900000,8.000000,1597.000000


## 4. Predictions

In [11]:
new_input = pd.DataFrame(np.array([[4.3, 1, 0.01, 1, 0.05, 5, 25, 0.9, 3, 0.6, 13]]),
                         columns=X_train.columns)
new_input

,fixed_acidity,volatile_acidity,citric_acid,residual_sugar,chlorides,free_sulfur_dioxide,total_sulfur_dioxide,density,pH,sulphates,alcohol
0,4.3,1.0,0.01,1.0,0.05,5.0,25.0,0.9,3.0,0.6,13.0


In [12]:
# new_input = np.array([[4.3, 1, 0.01, 1, 0.05, 5, 25, 0.9, 3, 0.6, 13]])

model.predict(new_input)[0]

5.7

## 5. Saving the model

In [13]:
pickle.dump(model, open('./backend/RFModel.pkl', 'wb'))

In [14]:
loaded_model = pickle.load(open('./backend/RFModel.pkl', 'rb'))

In [16]:
# testing model with new data
new_input = pd.DataFrame(np.array([[0, 0, 0, 0, 0, 0, 200, 0, 0, 0, 0]]),
                         columns=["fixed_acidity", "volatile_acidity", "citric_acid",
                                  "residual_sugar", "chlorides", "free_sulfur_dioxide",
                                   "total_sulfur_dioxide", "density",
                                   "pH", "sulphates", "alcohol"])


loaded_model.predict(new_input)[0]

5.5